In [1]:
import numpy as np
import matplotlib.pyplot as plt

class RadialBasisFunctionNetworks:
    '''Класс реализует сеть радиальнобазисных функций'''
    def __init__(self,input_shape,hidden_shape,output_shape,n=0.2,T=20,R = 4):
        self.input_shape = input_shape #Количество входных нейронов
        self.hidden_shape = hidden_shape #Количество скрытых нейронов
        self.output_shape = output_shape #Количество выходных нейронов
        
        self.weights = None
        self.centers = None
        self.sigma_mass = None #Ширина окна активационной функции
        
        self.n =n #Скорость обучения
        self.T =T #Коэффициент уменьшения n
        self.R =R #Количество близжайших соседей
        
        
    def _radial_basis_function(self,i_c,center,data_vector):
        '''Гауссова функция'''
        #np.linalg.norm(data_vector-center)**2
        sum = 0 
        for i in range(self.input_shape):
            sum+=(data_vector[i]-center[i])**2
        return np.exp(sum/(2* (self.sigma_mass[i_c])**2))
    
    def _calculate_interpolation_matrix(self,X):
        '''Вычисление интерполяционной матрицы'''
        print(X.shape[0])
        F = np.zeros((X.shape[0],self.hidden_shape))
        for ind_X,x in enumerate(X):
            for ind_c,center in enumerate(self.centers):
                F[ind_X,ind_c] = self._radial_basis_function(ind_c,center,x)
        return F
    
    def _calculate_pseudoinverse_matrix(self,F):
        '''Вычисление псевдоинверсной матрицы'''
        F_t = F.transpose()
        M_inv = np.linalg.inv((np.dot(F_t,F)))
        F_plus = np.dot(M_inv,F_t)
        return F_plus
    
    def _k_means(self,x):
        '''Алгоритм k-means для нахождения исходных центров данных'''

        #Уточнение центров скрытых нейронов
        random_index = np.random.permutation(x.shape[0]).tolist()
        self.centers = np.array([x[ind] for ind in random_index][:self.hidden_shape])
        for i,el in enumerate(x):
            d = np.array([])
            for j_c,c in enumerate(self.centers):
                d = np.append(d,(np.sum((el-c)**2))**(1/2))
            min = np.argmin(d)
            self.centers[min] = self.centers[min] + self.n*(el-self.centers[min])
            self.n = self.n/(1+(i/self.T))
        
        
        self.sigma_mass = np.array([])
        #Расчет сигма для скрытых нейронов (массив)
        for i in range(self.centers.shape[0]):
            c = self.centers[i]
            d = np.empty((0,2), float)
            for j in range(self.centers.shape[0]):
                if(j!=i):
                    s = (np.sum((c-self.centers[j])**2))**(1/2)
                    d = np.append(d, [[j,s]],axis = 0)
            d.view('i8,i8').sort(order=['f1'], axis=0)
            #R близжайших соседей
            d = d[:self.R]

            su = 0
            for r in range(d.shape[0]):
                su += (np.sum(self.centers[r]-c)**2)
            su = (su/self.R)**(1/2)
            self.sigma_mass = np.append(self.sigma_mass,[su])
        print("sigma_mass",self.sigma_mass)
    
    def fit(self,X,Y):
        '''Обучение'''
        #Случайно выбираем центры из начальных данных
#         random_index = np.random.permutation(X.shape[0]).tolist()
#         self.centers = [X[ind] for ind in random_index][:self.hidden_shape]
        self._k_means(X)
        F = self._calculate_interpolation_matrix(X)
        F_plus = self._calculate_pseudoinverse_matrix(F)
        self.weights = np.dot(F_plus,Y)
        
        
    def predict(self,X):
        '''Вычисляет по обучающей выборке вектор ответов Y'''
        F = self._calculate_interpolation_matrix(X)
        predictions = np.dot(F,self.weights)
        return predictions

In [2]:
#Чтение данных из файла
x=[]
#данные зависимости потребления Y (усл. ед.) от дохода X (усл.ед.) для некоторых домашних хозяйств. 
f = open("Irisy.txt", encoding = 'utf-8')
for line in f:
    l = line.split(' ')
    for i in range(len(l)):
        l[i] = float(l[i].strip(" ").lower().strip("\n"))
    x.append(l)
x = np.array(x)

In [3]:
y = []
f = open("y.txt", encoding = 'utf-8')
for line in f:
    l = line.split(' ')
    for i in range(len(l)):
        l[i] = float(l[i].strip(" ").lower().strip("\n"))
    y.append(l)
y = np.array(y)

In [4]:
#Считать x и y
model = RadialBasisFunctionNetworks(input_shape = 4,hidden_shape = 10,output_shape = 3)
model.fit(x,y)
y_pred = model.predict(x)

count = 0
for i,el in enumerate(y_pred):
    #print(np.argmax(el),np.argmax(y[i]))#по строкам
    if(np.argmax(el)==np.argmax(y[i])):
        count+=1

sigma_mass [2.68048503 1.54434452 1.56044865 2.59903828 1.54434452 4.4565938
 1.58271918 5.86800138 3.19139468 2.76315038]
150
150


In [5]:
print("Accuracy", (count/x.shape[0])*100)

Accuracy 96.0


# Тестирование сети для данных о кластеризации по вероятности страхового случая

In [6]:
#Чтение данных из файла
x=[]
#данные зависимости потребления Y (усл. ед.) от дохода X (усл.ед.) для некоторых домашних хозяйств. 
f = open("data_for_k_means.txt", encoding = 'utf-8')
for line in f:
    l = line.split(' ')
    for i in range(len(l)):
        l[i] = float(l[i].strip(" ").lower().strip("\n"))
    x.append(l)
x = np.array(x)


In [7]:
y = []
f = open("y_k_means.txt", encoding = 'utf-8')
for line in f:
    l = line.split(' ')
    for i in range(len(l)):
        l[i] = float(l[i].strip(" ").lower().strip("\n"))
    y.append(l)
y = np.array(y)

In [8]:
#Считать x и y
model2 = RadialBasisFunctionNetworks(input_shape = 4,hidden_shape = 10,output_shape = 4)
model2.fit(x,y)
y_pred = model2.predict(x)


count = 0
for i,el in enumerate(y_pred):
    #print(np.argmax(el),np.argmax(y[i]))#по строкам
    if(np.argmax(el)==np.argmax(y[i])):
        count+=1

sigma_mass [23.55700643 24.03269443 43.21615186 28.60181864 24.84404607 32.98455941
 40.24529868 72.28873948 35.81427574 35.80373138]
25
25


In [9]:
print("Accuracy", (count/x.shape[0])*100)

Accuracy 100.0


# Тестирование сети на данных из лабы Backpropagation

In [10]:
train_data = [
    ([2, 0, 0, 0], [0,0,1,0]),
    ([2,0,0,1], [0,0,1,0]),
    ([2,0,1,1], [1,0,0,0]),
    ([2,0,1,2], [1,0,0,0]),
    ([2,1,0,2], [0,0,0,1]),
    ([2,1,0,1], [1,0,0,0]),
    ([1,0,0,0], [0,0,1,0]),
    ([1,0,0,1], [0,0,0,1]),
    ([1,0,1,1], [1,0,0,0]),
    ([1,0,1,2], [0,0,0,1]),
    ([1,1,0,2], [0,0,0,1]),
    ([1,1,0,1], [0,0,0,1]),
    
    ([0,0,0,0], [0,0,1,0]),
    ([0,0,0,1], [0,0,0,1]),
    ([0,0,1,1], [0,0,0,1]),
    ([0,0,1,2], [0,1,0,0]),
    ([0,1,0,2], [0,1,0,0]),
    ([0,1,0,1], [0,0,0,1]),
]

test_data = [
    ([2, 1, 1, 1], [1,0,0,0]),
    ([1,1,1,2], [0,0,0,1]),
    ([0,0,0,0], [0,0,1,0]),
    ([0,1,1,1], [0,0,0,1]),
    ([2,0,1,3], [0,0,0,1]),
    ([2,1,0,3], [0,0,0,1]),
    ([0,1,0,3], [0,1,0,0]),
    
]

x,y = np.array([l for l,y in train_data]),np.array([y for l,y in train_data])
print(x)
x_test,y_test = np.array([l for l,y in test_data]),np.array([y for l,y in test_data])

[[2 0 0 0]
 [2 0 0 1]
 [2 0 1 1]
 [2 0 1 2]
 [2 1 0 2]
 [2 1 0 1]
 [1 0 0 0]
 [1 0 0 1]
 [1 0 1 1]
 [1 0 1 2]
 [1 1 0 2]
 [1 1 0 1]
 [0 0 0 0]
 [0 0 0 1]
 [0 0 1 1]
 [0 0 1 2]
 [0 1 0 2]
 [0 1 0 1]]


In [45]:
model3 = RadialBasisFunctionNetworks(input_shape = 4,hidden_shape = 15,output_shape = 4)
model3.fit(x,y)
y_pred = model3.predict(x)
#print(y_pred)

count = 0
for i,el in enumerate(y_pred):
    #print(np.argmax(el),np.argmax(y[i]))#по строкам
    if(np.argmax(el)==np.argmax(y[i])):
        count+=1
print("Accuracy", (count/x.shape[0])*100)

sigma_mass [2.59807621 1.5        1.5        1.5        1.32287566 1.5
 1.80277564 1.32287566 3.04138127 3.04138127 1.32287566 1.80277564
 2.17944947 1.32287566 1.5       ]
18
18
Accuracy 100.0


In [46]:
y_pred = model3.predict(x_test)

count = 0
for i,el in enumerate(y_pred):
    print(np.argmax(el),np.argmax(y_test[i]))#по строкам
    if(np.argmax(el)==np.argmax(y_test[i])):
        count+=1
print("Accuracy", (count/x_test.shape[0])*100)

7
0 0
3 3
2 2
3 3
1 3
1 3
1 1
Accuracy 71.42857142857143


In [8]:
n = 0.2
R = 4 #Количество близжайших соседей
T = 20

#Уточнение центров скрытых нейронов
random_index = np.random.permutation(x.shape[0]).tolist()
centers = np.array([x[ind] for ind in random_index][:20])
for i,el in enumerate(x):
    d = np.array([])
    for j_c,c in enumerate(centers):
        d = np.append(d,(np.sum((el-c)**2))**(1/2))
    min = np.argmin(d)
    centers[min] = centers[min] + n*(el-centers[min])
    n = n/(1+(i/T))
    print(i,n)
print(centers)


0 0.2
1 0.19047619047619047
2 0.17316017316017313
3 0.15057406361754186
4 0.12547838634795155
5 0.10038270907836125
6 0.07721746852181634
7 0.05719812483097506
8 0.040855803450696476
9 0.028176416172894122
10 0.018784277448596082
11 0.012118888676513602
12 0.007574305422821
13 0.004590488135043031
14 0.0027002871382606065
15 0.0015430212218632037
16 0.0008572340121462243
17 0.0004633697362952563
18 0.0002438788085764507
19 0.00012506605568023114
20 6.253302784011557e-05
21 3.0503916019568573e-05
22 1.4525674295032654e-05
23 6.7561275790849555e-06
24 3.0709670814022524e-06
25 1.3648742584010011e-06
26 5.93423590609131e-07
27 2.525206768549493e-07
28 1.0521694868956222e-07
29 4.294569334267845e-08
30 1.717827733707138e-08
31 6.73657934787113e-09
32 2.5909920568735113e-09
33 9.777328516503816e-10
34 3.621232783890302e-10
35 1.3168119214146553e-10
36 4.702899719338055e-11
37 1.6501402523993175e-11
38 5.690138801376957e-12
39 1.9288606106362565e-12
40 6.429535368787521e-13
41 2.108044383209

In [22]:
sigma_mass = np.array([])
#Расчет сигма для скрытых нейронов (массив)
for i in range(centers.shape[0]):
    c = centers[i]
    d = np.empty((0,2), float)
    for j in range(centers.shape[0]):
        if(j!=i):
            s = (np.sum((c-centers[j])**2))**(1/2)
            d = np.append(d, [[j,s]],axis = 0)
    
    d.view('i8,i8').sort(order=['f1'], axis=0)
    print(d)
    #R близжайших соседей
    d = d[:R]
    print(d)
    
    su = 0
    for r in range(d.shape[0]):
        su += (np.sum(centers[r]-c)**2)
    su/=R
    su = su**(1/2)
    print(c,su)
    sigma_mass = np.append(sigma_mass,[su])
print("sigma_mass",sigma_mass)

[[18.          0.46904158]
 [ 5.          0.71414284]
 [13.          0.78740079]
 [16.          0.83066239]
 [ 2.          1.03440804]
 [ 4.          1.04880885]
 [19.          1.21243557]
 [11.          1.22065556]
 [ 9.          1.2489996 ]
 [17.          1.92353841]
 [10.          2.16333077]
 [ 3.          2.47588368]
 [12.          2.57588447]
 [ 6.          3.04415197]
 [ 8.          3.08156055]
 [ 7.          3.08763494]
 [15.          3.08868904]
 [ 1.          3.09696418]
 [14.          3.26471292]]
[[18.          0.46904158]
 [ 5.          0.71414284]
 [13.          0.78740079]
 [16.          0.83066239]]
[5.6 3.  4.1 1.3] 2.932659505934289
[[ 6.          0.39653159]
 [ 8.          0.44852091]
 [12.          0.8062991 ]
 [ 7.          0.82817784]
 [14.          1.04232549]
 [15.          1.31368848]
 [13.          2.74994073]
 [18.          3.09448966]
 [ 0.          3.09696418]
 [16.          3.52230918]
 [ 2.          3.68987521]
 [ 5.          3.70445719]
 [ 4.          3.

In [43]:
a = np.array([1,2,3])
a = a-a
print(a)

[0 0 0]


In [141]:
A= [[1, 2, 3], [4, 5, 6]]
A = np.append(A, [[7, 8, 9]], axis=0)
print(A)

C = np.array([4, 5, 6])
C = np.append(C,[4, 5, 6])
print(C)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[4 5 6 4 5 6]


In [110]:
arr = np.empty((0,2), float)
arr = np.append(arr, np.array([[1,2]]), axis=0)
arr = np.append(arr, np.array([[4,5]]), axis=0)
print(arr)

[[1. 2.]
 [4. 5.]]


In [ ]:
l = [ 3.54188246e-06,  2.68118731e-08,  4.33050555e-05]
min(l)

In [ ]:
import matplotlib.pyplot as plt
x  = np.random.uniform(-1, 1, (10,3)) #равномерное распределение
print(x)

In [ ]:
import numpy as np
b = np.array([1,2,3,4,5,6])
c = np.array([1,1,1,1,1,1])
print(np.linalg.norm(b-c)**2)